In [2]:
import requests as req

In [25]:
import base64
from pydub import AudioSegment
import io

def save_audio_data_to_mp3(data, output_filename="output.mp3"):
    """
    Giải mã dữ liệu audio từ cấu trúc JSON và lưu thành file MP3.

    Args:
        data (dict): Đối tượng JSON chứa dữ liệu audio.
        output_filename (str): Tên file MP3 đầu ra.
    """
    try:
        # 1. Trích xuất dữ liệu Base64 và thông tin audio
        audio_part = data['candidates'][0]['content']['parts'][0]['inlineData']
        base64_data = audio_part['data']
        mime_type = audio_part['mimeType'] # e.g., 'audio/L16;codec=pcm;rate=24000'

        # Kiểm tra xem có dữ liệu để xử lý không
        if not base64_data:
            print("Lỗi: Dữ liệu audio trống.")
            return

        # 2. Giải mã Base64 để lấy dữ liệu audio thô (raw bytes)
        raw_audio_data = base64.b64decode(base64_data)

        # 3. Phân tích thông tin từ mimeType để cấu hình pydub
        # Giả định định dạng là PCM 16-bit (2 bytes) và 1 kênh (mono)
        sample_rate = int(mime_type.split('rate=')[1])
        sample_width = 2  # 16 bits = 2 bytes
        channels = 1      # Mono

        # 4. Tạo một đối tượng AudioSegment từ dữ liệu thô
        audio_segment = AudioSegment(
            data=raw_audio_data,
            sample_width=sample_width,
            frame_rate=sample_rate,
            channels=channels
        )

        # 5. Xuất file MP3
        print(f"Đang xuất file âm thanh tới '{output_filename}'...")
        audio_segment.export(output_filename, format="wav")
        print("Hoàn thành! File đã được lưu thành công.")

    except (KeyError, IndexError) as e:
        print(f"Lỗi: Cấu trúc JSON không hợp lệ hoặc không tìm thấy dữ liệu audio. Chi tiết: {e}")
    except Exception as e:
        print(f"Đã xảy ra lỗi không mong muốn: {e}")
        print("Vui lòng đảm bảo bạn đã cài đặt 'pydub' và 'ffmpeg'.")
        print("Để cài đặt ffmpeg, hãy truy cập: https://ffmpeg.org/download.html")

In [ ]:
from typing import List, Dict
import json
def create_conversation(content: List[Dict[str, str]]) -> None:
    api_url = "https://api.thucchien.ai/gemini/v1beta/models/gemini-2.5-flash-preview-tts:generateContent"
    headers = {
        'x-goog-api-key': 'sk-_L5p4n0oPzdMhkMFZh8clA',
        'Content-Type': 'application/json'
    }

    data = json.dumps("""{
    "contents": {content},
    "generationConfig": {
        "responseModalities": ["AUDIO"],
        "speechConfig": {
            "voiceConfig": {
            "prebuiltVoiceConfig": {
                "voiceName": "Kore"
            }
            }
        }
    }
    }""".format(content=content))

    res = req.post(api_url, data=data, headers=headers, stream=True)

    save_audio_data_to_mp3(res.json(), "out.wav")


    

In [ ]:
create_conversation